In [1026]:
import pickle
import numpy as np
from ctypes import *
from scipy.interpolate import LinearNDInterpolator, interp1d
from obspy import UTCDateTime
import obspy
from matplotlib import pyplot as plt
from scipy.ndimage import uniform_filter1d
from scipy.stats import median_abs_deviation
from obspy.signal.trigger import classic_sta_lta
import bisect
import os
import concurrent.futures
from itertools import repeat, product
from wpca import WPCA
from scipy.spatial.distance import cdist, euclidean
from skimage.transform import radon, iradon
import pygmt
import xarray
from scipy.signal import find_peaks, lfilter, iirfilter, zpk2sos, sosfiltfilt, savgol_filter
import SPECFEM3D_interface
import numba
import cmcrameri.cm as cmc

In [2]:
def calc_times(source_x, source_z, vp, vs, x_m, z_m, ind):
    so_file = "/home/moshe/eyal/eikonal/Eikonal3D.so"
    eikonal_3d = CDLL(so_file)
    size_x = np.ma.size(vp, axis=0)
    size_y = 1
    size_z = np.ma.size(vp, axis=1)
    vp_flat = np.ndarray.flatten(vp, 'C')
    vs_flat = np.ndarray.flatten(vs, 'C')
    v_dim = [size_x, size_y, size_z]
    d_elem = [x_m/(size_x-1), 250, z_m/(size_z-1)]
    source = [int(source_x/(x_m/(size_x-1))), 0, int(source_z/(z_m/(size_z-1)))]
    print(source)

    eikonal_3d_func = eikonal_3d._Z9eikonal3DPfPiS_S0_S_
    eikonal_3d_func.restype = None
    vp_flatc = (c_float * len(vp_flat))(*vp_flat)
    vs_flatc = (c_float * len(vs_flat))(*vs_flat)
    vDim = (c_int * len(v_dim))(*v_dim)
    dElem = (c_float * len(d_elem))(*d_elem)
    sourceC = (c_int * len(source))(*source)

    times_vp = (c_float * len(vp_flat))()
    eikonal_3d_func(vp_flatc, vDim, dElem, sourceC, times_vp)
    times_vp = np.array(times_vp[:])
    times_vp = times_vp.reshape((size_z, size_y, size_x))
    times_vp = times_vp.transpose((2, 1, 0))

    times_vs = (c_float * len(vs_flat))()
    eikonal_3d_func(vs_flatc, vDim, dElem, sourceC, times_vs)
    times_vs = np.array(times_vs[:])
    times_vs = times_vs.reshape((size_z, size_y, size_x))
    times_vs = times_vs.transpose((2, 1, 0))

    return ind, times_vp, times_vs

In [386]:
def get_velocity_models(size):
    tomographic_models = SPECFEM3D_interface.read_tomographic_models("mtinv", 0)
    ind = 0
    vp = np.reshape(tomographic_models[:, 2], (size+1, size+1), order='F')
    vs = np.reshape(tomographic_models[:, 3], (size+1, size+1), order='F')
    return vp[::4, ::4], vs[::4, ::4]

In [387]:
v_p, v_s = get_velocity_models(320)

In [5]:
x_m = 20000
z_m = 20000

In [389]:
import data
import re

stations = data.get_stations()
fibres = ["BI", "BII", "S"]

stations_eikonal_tables_vp = []
stations_eikonal_tables_vs = []

for fibre in fibres:
    fibre_stations = [station for station in stations if re.sub(r'[^a-zA-Z]', '', station.station) == fibre][::250]
    stations_x_s = [station.longitude for station in fibre_stations]
    stations_z_s = [20000 - station.burial for station in fibre_stations]
    indices = list(range(len(stations_x_s)))
    stations_eikonal_tables_vp_s = np.empty((81, 81, len(stations_x_s)))
    stations_eikonal_tables_vs_s = np.empty((81, 81, len(stations_x_s)))
    for i in range(len(stations_x_s)):
        i, eikonal_fibre_vp_curr, eikonal_fibre_vs_curr = calc_times(stations_x_s[i], stations_z_s[i], v_p, v_s, x_m, z_m, 
                                                                     indices[i])
        stations_eikonal_tables_vp_s[:, :, i] = eikonal_fibre_vp_curr[:, 0, :]
        stations_eikonal_tables_vs_s[:, :, i] = eikonal_fibre_vs_curr[:, 0, :]
    
    fibre_stations = [station for station in stations if re.sub(r'[^a-zA-Z]', '', station.station) == fibre][::5]
    stations_x = [station.longitude for station in fibre_stations]
    stations_z = [20000 - station.burial for station in fibre_stations]
    stations_eikonal_tables_vp_curr = np.empty((81, 81, len(stations_x)))
    stations_eikonal_tables_vs_curr = np.empty((81, 81, len(stations_x)))
    for i in range(stations_eikonal_tables_vp_s.shape[0]):
        for j in range(stations_eikonal_tables_vp_s.shape[1]):
            if fibre == "S":
                vp_interp = interp1d(stations_x_s, stations_eikonal_tables_vp_s[i, j, :])
                stations_eikonal_tables_vp_curr[i, j, :] = vp_interp(stations_x)
                vs_interp = interp1d(stations_x_s, stations_eikonal_tables_vs_s[i, j, :])
                stations_eikonal_tables_vs_curr[i, j, :] = vs_interp(stations_x)
            else:
                vp_interp = interp1d(stations_z_s, stations_eikonal_tables_vp_s[i, j, :])
                stations_eikonal_tables_vp_curr[i, j, :] = vp_interp(stations_z)
                vs_interp = interp1d(stations_z_s, stations_eikonal_tables_vs_s[i, j, :])
                stations_eikonal_tables_vs_curr[i, j, :] = vs_interp(stations_z)
    stations_eikonal_tables_vp.append(stations_eikonal_tables_vp_curr)
    stations_eikonal_tables_vs.append(stations_eikonal_tables_vs_curr)
stations_eikonal_tables_vp = np.concatenate(stations_eikonal_tables_vp, axis=2)
stations_eikonal_tables_vs = np.concatenate(stations_eikonal_tables_vs, axis=2)

[14, 0, 80]
[14, 0, 79]
[14, 0, 78]
[14, 0, 77]
[14, 0, 76]
[14, 0, 75]
[14, 0, 74]
[14, 0, 73]
[14, 0, 72]
[62, 0, 80]
[62, 0, 79]
[62, 0, 78]
[62, 0, 77]
[62, 0, 76]
[62, 0, 75]
[62, 0, 74]
[62, 0, 73]
[62, 0, 72]
[4, 0, 80]
[5, 0, 80]
[6, 0, 80]
[7, 0, 80]
[8, 0, 80]
[9, 0, 80]
[10, 0, 80]
[11, 0, 80]
[12, 0, 80]
[13, 0, 80]
[14, 0, 80]
[15, 0, 80]
[16, 0, 80]
[17, 0, 80]
[18, 0, 80]
[19, 0, 80]
[20, 0, 80]
[21, 0, 80]
[22, 0, 80]
[23, 0, 80]
[24, 0, 80]
[25, 0, 80]
[26, 0, 80]
[27, 0, 80]
[28, 0, 80]
[29, 0, 80]
[30, 0, 80]
[31, 0, 80]
[32, 0, 80]
[33, 0, 80]
[34, 0, 80]
[35, 0, 80]
[36, 0, 80]
[37, 0, 80]
[38, 0, 80]
[39, 0, 80]
[40, 0, 80]
[41, 0, 80]
[42, 0, 80]
[43, 0, 80]
[44, 0, 80]
[45, 0, 80]
[46, 0, 80]
[47, 0, 80]
[48, 0, 80]
[49, 0, 80]
[50, 0, 80]
[51, 0, 80]
[52, 0, 80]
[53, 0, 80]
[54, 0, 80]
[55, 0, 80]
[56, 0, 80]
[57, 0, 80]
[58, 0, 80]
[59, 0, 80]
[60, 0, 80]
[61, 0, 80]
[62, 0, 80]
[63, 0, 80]
[64, 0, 80]
[65, 0, 80]
[66, 0, 80]
[67, 0, 80]
[68, 0, 80]
[69, 0, 80

In [75]:
dx = 5
min_spat_freq = 0.01 / 2000
max_spat_freq = 10 / 2000
space_sampling_rate = 1 / dx
space_nyquist = space_sampling_rate / 2
space_low = min_spat_freq / space_nyquist
space_high = max_spat_freq / space_nyquist
z_space, p_space, k_space = iirfilter(4, [space_low, space_high], btype='band', ftype='butter', output='zpk')
sos_space = zpk2sos(z_space, p_space, k_space)

In [1251]:
def calc_locations(i):
    with open(f"/DATA/eyal/specfem2d/mtinv/observed_seismograms{i:04d}.pk", "rb") as f:
        obs1 = pickle.load(f)
    obs1.filter("lowpass", freq=10)
    waveform_array = np.array([trace.data for trace in obs1])
    waveform_array_ext = np.concatenate((np.zeros(waveform_array.shape), waveform_array), axis=1)
    waveform_array_ext /= np.max(np.abs(waveform_array_ext), axis=1, keepdims=True)
    score, std_score, mad_score = calc_single_slice(np.abs(hilbert(waveform_array_ext, axis=1))[:802,:], dt, dims, 
                                                    stations_eikonal_tables_vp[:, :, :802], 
                                                    stations_eikonal_tables_vs[:, :, :802], list(range(401)), 
                                                    list(range(401, 802)))
    comb_score = np.std(mad_score[mad_score < 1000]) / np.std(score[score > 0]) / 4 * score - mad_score
    probability = score_to_probability(comb_score)
    depth_ind = find_closest_index(np.cumsum(np.sum(probability, axis=0)), 0.5)
    if np.sum(probability, axis=0)[depth_ind] / np.sum(probability, axis=0).max() < 0.33:
        depth_ind = np.argmax(np.sum(probability, axis=0))
    scores_s, mad_scores_s, std_scores_s = calc_scores_reduced(np.abs(hilbert(waveform_array_ext, axis=1)), dt, dims, 
                                                               stations_eikonal_tables_vp, stations_eikonal_tables_vs, 
                                                               depth_ind, list(range(401)), list(range(401, 802)), 
                                                               list(range(802, 4402)))
    comb_score_s = np.std(mad_scores_s[mad_scores_s < 1000]) / np.std(scores_s[scores_s > 0]) * scores_s - mad_scores_s
    probability_s = score_to_probability(comb_score_s)
    spat_ind = find_closest_index(np.cumsum(probability_s), 0.5)
    if probability_s[spat_ind] / probability_s.max() < 0.33:
        spat_ind = np.argmax(probability_s)
    loc = [spat_ind, depth_ind]
    return i, score, std_score, mad_score, comb_score, probability, scores_s, mad_scores_s, std_scores_s, comb_score_s, probability_s, loc

In [1252]:
score_all = {}
std_score_all = {}
mad_score_all = {}
comb_score_all = {}
probability_all = {}
scores_s_all = {}
mad_scores_s_all = {}
std_scores_s_all = {}
comb_score_s_all = {}
probability_s_all = {}
loc_all = {}
inds = list(range(1, 21))

with concurrent.futures.ProcessPoolExecutor(max_workers=20) as executor:
    for i, score, std_score, mad_score, comb_score, probability, scores_s, mad_scores_s, std_scores_s, comb_score_s, probability_s, loc in executor.map(calc_locations, inds):
        score_all[i] = score
        std_score_all[i] = std_score
        mad_score_all[i] = mad_score
        comb_score_all[i] = comb_score
        probability_all[i] = probability
        scores_s_all[i] = scores_s
        mad_scores_s_all[i] = mad_scores_s
        std_scores_s_all[i] = std_scores_s
        comb_score_s_all[i] = comb_score_s
        probability_s_all[i] = probability_s
        loc_all[i] = loc

In [856]:
for i in range(1, 21)
    with open("/DATA/eyal/specfem2d/mtinv/observed_seismograms{i:04d}.pk", "rb") as f:
        obs1 = pickle.load(f)
    obs1.filter("lowpass", freq=10)
    waveform_array = np.array([trace.data for trace in obs1])
    waveform_array_ext = np.concatenate((np.zeros(waveform_array.shape), waveform_array), axis=1)
    waveform_array_ext /= np.max(np.abs(waveform_array_ext), axis=1, keepdims=True)
    score, std_score, mad_score = calc_single_slice(np.abs(hilbert(waveform_array_ext, axis=1))[:802,:], dt, dims, 
                                                    stations_eikonal_tables_vp[:, :, :802], 
                                                    stations_eikonal_tables_vs[:, :, :802], list(range(401)), 
                                                    list(range(401, 802)))
    comb_score = np.std(mad_score[mad_score < 1000]) / np.std(score[score > 0]) / 4 * score - mad_score
    probability = score_to_probability(comb_score)
    depth_ind = find_closest_index(np.cumsum(np.sum(probability, axis=0)), 0.5)
    scores_s, mad_scores_s, std_scores_s = calc_scores_reduced(np.abs(hilbert(waveform_array_ext, axis=1)), dt, dims, 
                                                               stations_eikonal_tables_vp, stations_eikonal_tables_vs, 
                                                               depth_ind, list(range(401)), list(range(401, 802)), 
                                                               list(range(802, 4402)))
    comb_score_s = np.std(mad_scores_s[mad_scores_s < 1000]) / np.std(scores_s[scores_s > 0]) * scores_s - mad_scores_s
    probability_s = score_to_probability(comb_score_s)
    loc = [probability_s.argmax(), depth_ind]

In [147]:
def calc_single_shifted_waveform_clean(waveform_array, dt, stations_eikonal_tables_vp, stations_eikonal_tables_vs):
    shifted_waveform_array_p = np.zeros(waveform_array.shape)
    shifted_waveform_array_s = np.zeros(waveform_array.shape)

    for l in range(np.ma.size(waveform_array, axis=0)):
        shifted_waveform_array_p[l, :] = \
            np.roll(waveform_array[l, :],
                    -int(np.round(stations_eikonal_tables_vp[l] / dt)))
        shifted_waveform_array_s[l, :] = \
            np.roll(waveform_array[l, :],
                    -int(np.round(stations_eikonal_tables_vs[l] / dt)))

    return shifted_waveform_array_p, shifted_waveform_array_s

In [40]:
def get_s_index(subarray_sum):
    s_sus_p_max = int(np.where(subarray_sum > subarray_sum.max()/5)[0][0] + 0.5/dt)
    if s_sus_p_max >= len(subarray_sum):
        return subarray_sum.argmax()
    s_sus_s_max = subarray_sum[s_sus_p_max:].max()
    s_sus_s_argmax = subarray_sum[s_sus_p_max:].argmax() + s_sus_p_max
    if s_sus_p_max == s_sus_s_argmax:
        return subarray_sum.argmax()
    if s_sus_s_max / subarray_sum.max() > 0.5 and subarray_sum[s_sus_p_max:s_sus_s_argmax].min() / s_sus_s_max < 0.5:
        return s_sus_s_argmax
    else:
        return subarray_sum.argmax()

In [63]:
@numba.njit(fastmath=True, cache=True)
def shift_data(waveform_array, stations_eikonal_tables_vp, stations_eikonal_tables_vs, inv_dt, num_stations, num_samples):
    shifted_waveform_array_p = np.empty((num_stations, num_samples), dtype=waveform_array.dtype)
    shifted_waveform_array_s = np.empty((num_stations, num_samples), dtype=waveform_array.dtype)

    for l in range(num_stations):
        # Calculate integer shifts for this station
        shift_p = -int(np.round(stations_eikonal_tables_vp[l] * inv_dt))
        shift_s = -int(np.round(stations_eikonal_tables_vs[l] * inv_dt))

        # Apply roll manually using modulo indexing
        for sample_idx in range(num_samples):
            # Calculate source index based on the desired shift
            src_idx_p = (sample_idx - shift_p) % num_samples
            src_idx_s = (sample_idx - shift_s) % num_samples
            # Assign value from source index to current target index
            shifted_waveform_array_p[l, sample_idx] = waveform_array[l, src_idx_p]
            shifted_waveform_array_s[l, sample_idx] = waveform_array[l, src_idx_s]
    return shifted_waveform_array_p, shifted_waveform_array_s

In [1234]:
def calc_single_slice(waveform_array, dt, dims, stations_eikonal_tables_vp, stations_eikonal_tables_vs, inds_a1, inds_a2):
    score = np.zeros((dims[0], dims[1]))
    std_score = np.zeros((dims[0], dims[1]))
    mad_score = np.zeros((dims[0], dims[1]))
    num_stations = np.ma.size(waveform_array, axis=0)
    num_samples = np.ma.size(waveform_array, axis=1)
    waveform_array /= np.max(np.abs(waveform_array), axis=1, keepdims=True)

    for i in range(dims[0]):
        for j in range(dims[1]):
            shifted_waveform_array_p, shifted_waveform_array_s = shift_data(waveform_array, stations_eikonal_tables_vp[i, j, :], 
                                                                            stations_eikonal_tables_vs[i, j, :], 1/dt, num_stations, num_samples)

            shifted_waveform_array_p_a1 = shifted_waveform_array_p[inds_a1]
            shifted_waveform_array_s_a1 = shifted_waveform_array_s[inds_a1]
            shifted_waveform_array_p_a2 = shifted_waveform_array_p[inds_a2]
            shifted_waveform_array_s_a2 = shifted_waveform_array_s[inds_a2]
            
            p_a1_sum = savgol_filter(np.abs(shifted_waveform_array_p_a1.sum(axis=0) / len(inds_a1)), int(0.3/dt), 4)
            s_a1_sum = savgol_filter(np.abs(shifted_waveform_array_s_a1.sum(axis=0) / len(inds_a1)), int(0.3/dt), 4)
            p_a2_sum = savgol_filter(np.abs(shifted_waveform_array_p_a2.sum(axis=0) / len(inds_a2)), int(0.3/dt), 4)
            s_a2_sum = savgol_filter(np.abs(shifted_waveform_array_s_a2.sum(axis=0) / len(inds_a2)), int(0.3/dt), 4)

            innds_pa1, _ = find_peaks(p_a1_sum, height=0.11, distance=10)
            innds_sa1, _ = find_peaks(s_a1_sum, height=0.5, distance=10)
            innds_pa2, _ = find_peaks(p_a2_sum, height=0.11, distance=10)
            innds_sa2, _ = find_peaks(s_a2_sum, height=0.5, distance=10)

            
            if len(innds_pa1) == 0 or len(innds_sa1) == 0 or len(innds_pa2) == 0 or len(innds_sa2) == 0:
                score[i, j] = 0
                mad_score[i, j] = 1000
                std_score[i, j] = 1000
                continue
                
            p_a1_max_ind = innds_pa1[0]
            max_peak = 0
            for peak in innds_sa1:
                if s_a1_sum[peak] > max_peak:
                    s_a1_max_ind = peak
            p_a2_max_ind = innds_pa2[0]
            for peak in innds_sa2:
                if s_a2_sum[peak] > max_peak:
                    s_a2_max_ind = peak

            p_a1_max = p_a1_sum[p_a1_max_ind]
            s_a1_max = s_a1_sum[s_a1_max_ind]
            p_a2_max = p_a2_sum[p_a2_max_ind]
            s_a2_max = s_a2_sum[s_a2_max_ind]
            
            score[i, j] = p_a1_max + s_a1_max + p_a2_max + s_a2_max

            mad_score[i, j] = median_abs_deviation([p_a1_max_ind, s_a1_max_ind, p_a2_max_ind, s_a2_max_ind])

            std_score[i, j] = np.std([p_a1_max_ind, s_a1_max_ind, p_a2_max_ind, s_a2_max_ind])
            
    return score, std_score, mad_score

In [1236]:
def calc_scores_reduced(waveform_array, dt, dims, stations_eikonal_tables_vp, stations_eikonal_tables_vs, j, inds_a1, inds_a2, inds_a3):
    score = np.zeros((dims[0]))
    mad_score = np.zeros((dims[0]))
    std_score = np.zeros((dims[0]))
    num_stations = np.ma.size(waveform_array, axis=0)
    num_samples = np.ma.size(waveform_array, axis=1)
    waveform_array /= np.max(np.abs(waveform_array), axis=1, keepdims=True)

    for i in range(dims[0]):
        shifted_waveform_array_p, shifted_waveform_array_s = shift_data(waveform_array, stations_eikonal_tables_vp[i, j, :], 
                                                                        stations_eikonal_tables_vs[i, j, :], 1/dt, num_stations, num_samples)
        
        shifted_waveform_array_p_a1 = shifted_waveform_array_p[inds_a1]
        shifted_waveform_array_s_a1 = shifted_waveform_array_s[inds_a1]
        shifted_waveform_array_p_a2 = shifted_waveform_array_p[inds_a2]
        shifted_waveform_array_s_a2 = shifted_waveform_array_s[inds_a2]
        shifted_waveform_array_s_a3 = shifted_waveform_array_s[inds_a3]

        p_a1_sum = np.abs(shifted_waveform_array_p_a1.sum(axis=0)) / (len(inds_a1) + len(inds_a2) + len(inds_a3))
        s_a1_sum = np.abs(shifted_waveform_array_s_a1.sum(axis=0)) / (len(inds_a1) + len(inds_a2) + len(inds_a3))
        p_a2_sum = np.abs(shifted_waveform_array_p_a2.sum(axis=0)) / (len(inds_a1) + len(inds_a2) + len(inds_a3))
        s_a2_sum = np.abs(shifted_waveform_array_s_a2.sum(axis=0)) / (len(inds_a1) + len(inds_a2) + len(inds_a3))
        s_a3_sum = np.abs(shifted_waveform_array_s_a3.sum(axis=0)) / (len(inds_a1) + len(inds_a2) + len(inds_a3))

        innds_pa1, _ = find_peaks(p_a1_sum, height=0.11 * len(inds_a1) / (len(inds_a1) + len(inds_a2) + len(inds_a3)))
        innds_sa1, _ = find_peaks(s_a1_sum, height=0.5 * len(inds_a1) / (len(inds_a1) + len(inds_a2) + len(inds_a3)))
        innds_pa2, _ = find_peaks(p_a2_sum, height=0.11 * len(inds_a2) / (len(inds_a1) + len(inds_a2) + len(inds_a3)))
        innds_sa2, _ = find_peaks(s_a2_sum, height=0.5 * len(inds_a2) / (len(inds_a1) + len(inds_a2) + len(inds_a3)))
        innds_sa3, _ = find_peaks(s_a3_sum, height=0.5 * len(inds_a3) / (len(inds_a1) + len(inds_a2) + len(inds_a3)))

        if len(innds_pa1) == 0 or len(innds_sa1) == 0 or len(innds_pa2) == 0 or len(innds_sa2) == 0 or len(innds_sa3) == 0:
            score[i] = 0
            mad_score[i] = 1000
            std_score[i] = 1000
            continue

        p_a1_max_ind = innds_pa1[0]
        s_a1_max_ind = innds_sa1[0]
        p_a2_max_ind = innds_pa2[0]
        s_a2_max_ind = innds_sa2[0]
        s_a3_max_ind = innds_sa3[0]

        p_a1_max = p_a1_sum[p_a1_max_ind]
        s_a1_max = s_a1_sum[s_a1_max_ind]
        p_a2_max = p_a2_sum[p_a2_max_ind]
        s_a2_max = s_a2_sum[s_a2_max_ind]
        s_a3_max = s_a3_sum[s_a3_max_ind]

        score[i] = p_a1_max + s_a1_max + p_a2_max + s_a2_max + s_a3_max

        mad_score[i] = median_abs_deviation([p_a1_max_ind, s_a1_max_ind, p_a2_max_ind, s_a2_max_ind, s_a3_max_ind])

        std_score[i] = np.std([p_a1_max_ind, s_a1_max_ind, p_a2_max_ind, s_a2_max_ind, s_a3_max_ind])
            
    return score, mad_score, std_score

In [837]:
def find_closest_index(arr: np.ndarray, value: float) -> int:
    """
    Finds the index of the element in a sorted NumPy array that is closest
    in value to the requested 'value'.

    Args:
        arr (np.ndarray): A 1D NumPy array that must be sorted in ascending order.
        value (float): The value for which to find the closest element's index.

    Returns:
        int: The index of the element in 'arr' that is closest to 'value'.
             In case of a tie (value is equidistant from two elements),
             it defaults to the larger element's index.
    """
    if not np.all(arr[:-1] <= arr[1:]): # Basic check for sorted array
        raise ValueError("Input array must be sorted in ascending order.")

    # Find the index where 'value' would be inserted to maintain sorted order.
    # 'side='left'' means it returns the first index 'i' such that arr[i] >= value.
    idx = np.searchsorted(arr, value, side='left')

    # Handle edge cases where 'value' is outside the array's range
    if idx == 0:
        # 'value' is less than or equal to the first element, so the first element is closest.
        return 0
    elif idx == len(arr):
        # 'value' is greater than the last element, so the last element is closest.
        return len(arr) - 1
    else:
        # 'value' falls between arr[idx-1] and arr[idx]
        # Compare the absolute difference to find which is closer
        diff_left = abs(value - arr[idx-1])
        diff_right = abs(value - arr[idx])

        if diff_left < diff_right:
            return idx - 1  # arr[idx-1] is closer
        else:
            return idx      # arr[idx] is closer or equidistant (defaults to right)

In [829]:
def score_to_probability(scores_array):
    prob_array = scores_array.copy()
    prob_array -= prob_array[prob_array>-1000].min()
    wanted_half_val = prob_array.max() / 40
    prob_array -= prob_array.max()
    prob_array /= (wanted_half_val / np.log(2 * np.e - 1))
    prob_array[prob_array < -300] = -300
    prob_array = 1 / (1 + np.exp(-prob_array))
    prob_array /= prob_array.sum()

    return prob_array

In [781]:
def estimate_geometric_median(probability, eps=1e-5):
    points, weights = probability_to_point_list_prob(probability)
    y = np.mean(points, 0)

    while True:
        D = cdist(points, [y])
        nonzeros = (D != 0)[:, 0]
        zeros = (D == 0)[:, 0]
        Dinv = weights[nonzeros] / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * points[nonzeros], 0)
        num_zeros = len(points) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(points):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            eta = 0 if num_zeros == 0 else weights[zeros][0]
            rinv = 0 if r == 0 else eta / r
            y1 = max(0, 1 - rinv) * T + min(1, rinv) * y
        if euclidean(y, y1) < eps:
            return y1
        y = y1
    return y

In [782]:
def probability_to_point_list_prob(probability):
    xs = list(range(np.ma.size(probability, axis=0)))
    ys = list(range(np.ma.size(probability, axis=1)))
    zs = list(range(np.ma.size(probability, axis=2)))

    point_list = np.asarray(list(product(xs, ys, zs)))
    probability_1d = probability.ravel()
    probability_1d = probability_1d.reshape((np.ma.size(point_list, axis=0), 1))

    return point_list, probability_1d

In [783]:
def calc_confidence_ellipsoid(probability):
    points, weights = probability_to_point_list_prob(probability)
    weights_ext = np.squeeze(np.dstack([weights, weights, weights]))

    pca = WPCA().fit(points, weights=weights_ext)
    ellipsoid_orientation = pca.components_
    pca.explained_variance_[pca.explained_variance_ < 0] = 0
    half_uncertainties = np.sqrt(7.815 * pca.explained_variance_)

    return half_uncertainties, ellipsoid_orientation